in this notebook, I combined the data that is needed for our research project, using 76 countries in GPS as the key because this is our main interest in the project, all the data has been filtered to at least after 2003, for the reason that democratic index only starts from 2006.
notice:
- democratic index range from 2006 to 2019, 2007 & 2009 doesn't exist
- Bosnia Herzegovina have no record in humanitarian aids data.
- 54 countries do not have ODA record
- 63 countries do not have aid received record
- government expense is as GDP % but in current and constant value wasn't specified and the data looks fishy  

### data output 
| Variable     |      Content                                                   |
|:-------------|:------------------------------------------------------------- :|
| country| name of countries from Global Preference Survey(GPS) data            |
| isocode | isocode of the countries                             |
|demo2019 ~ demo2006| democratic index by year|
|altruism| altruism score from GPS | 
|gdp2003 ~ gdp2019| gdp in current US dollar by year|
|funding2003 ~ funding2019| humanitarian aid fundinn in US dollar by year       |
|pledge2003 ~ pledge2019|pledging humanitarian aid in US dollar by year        |
|funding2003_gdp ~ funding2019_gdp| funding / gdp the year * 100       |
|gdpcapita2003 ~ gdpcapita2019| gdp per capita in current US dollar by year  |
|govexpense2003 ~ govexpense2019| General government total expenditure as % of GDP  |
|region| region group from WDI data|
|income_type| income group from WDI data|
|pop2003 ~ pop2019| population data by year |
|oda2003 ~ oda2019|Net Official development assistance in current US dollar |






In [5]:
import pandas as pd
import glob
from auxiliary import *
pd.set_option('display.max_columns', 500)



### read in  democracy index from The Economist's Democracy Index

In [9]:
democracy = pd.read_excel('data/EIU_Democracy_Index_2006_to_2019.xlsx').rename(columns={"Unnamed: 0": "country"})

In [10]:
country_dict = {'US': 'United States',  'Bosnia and Hercegovina' : 'Bosnia Herzegovina', 'UK' : 'United Kingdom', 'UAE': 'United Arab Emirates' }
democracy = democracy.replace({"country": country_dict})
democracy = democracy.set_index('country').add_prefix('demo').reset_index()
democracy.head()

,country,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006
0,Canada,9.22,9.15,9.15,9.15,9.08,9.08,9.08,9.08,9.08,9.08,9.07,9.07
1,United States,7.96,7.96,7.98,7.98,8.05,8.11,8.11,8.11,8.11,8.18,8.22,8.22
2,Austria,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69
3,Belgium,7.64,7.78,7.78,7.77,7.93,7.93,8.05,8.05,8.05,8.05,8.16,8.15
4,Cyprus,7.59,7.59,7.59,7.65,7.53,7.40,7.29,7.29,7.29,7.29,7.70,7.60


### read in Global Preference Survey (GPS) data and merge with  democracy index 

In [11]:
gps = pd.read_stata('data/country.dta')
gps.shape

(76, 8)

In [12]:
gps.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628


In [13]:
gps_democracy = pd.merge(gps, democracy, how = 'left', on = 'country')
gps_democracy.shape

(76, 20)

In [14]:
# gps_democracy[gps_democracy.isna().any(axis=1)]
gps_democracy.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69


### read in UN Humanitarian Affairs Financial Tracking Service data by year and rename columns for merging
### notice Bosnia does not get matched

In [15]:
file_name = 'data/OCHA_FTS_Government_Donations_20'
extension = '.xlsx'
all_filenames = [i for i in glob.glob(f'{file_name}*{extension}')]

In [16]:
for i in range(len(all_filenames)):
    new = pd.read_excel(all_filenames[i], sheet_name='Export data', skiprows=2).rename(columns={"Source org.": "country", "Funding US$": "funding", "Pledges US$" : "pledge"})
    if i == 0:
        df = new
    elif i < 7:
        df = pd.merge(df, new, how='outer', on="country", suffixes=('', f'200{i+3}'))       
    else:
        df = pd.merge(df, new, how='outer', on="country", suffixes=('', f'20{i+3}'))

donate = df.rename(columns={"pledge": "pledge2003", "funding": "funding2003"})

donate['country'] = donate['country'].str.rstrip(' Government of')
donate['country'] = donate['country'].str.rstrip(',')


In [17]:
#checking 2006 & 2020 manually 
country_dict = {'United States of America': 'United States',  'Saudi Arabia (Kingdom of)' : 'Saudi Arabia', 'Russian Federation' : 'Russia', 'Korea, Republic of': 'South Korea', 'Viet Nam' : 'Vietnam'}
donate = donate.replace({"country": country_dict})

In [18]:
gps_demo_donate = pd.merge(gps_democracy, donate,  how = 'left', on = 'country')
gps_demo_donate.shape

(76, 56)

In [19]:
gps_demo_donate.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,0.0,NaN,NaN,NaN,NaN,24000000.0,0.0,136493.0,0.0,2379910.0,0.0,3360.0,0.0,3000.0,0.0,2000.0,0.0,NaN,NaN,1000.0,0.0,NaN,NaN,NaN,NaN,3870542.0,0.0,903177.0,0.0,NaN,NaN
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,0.0,NaN,NaN,14769749.0,0.0,NaN,NaN,10000.0,0.0,10000.0,0.0,76432.0,0.0,10110696.0,0.0,16010000.0,7000000.0,NaN,NaN,2000000.0,0.0,10000.0,0.0,0.0,0.0,660000.0,0.0,10000.0,0.0,10000.0,0.0,NaN,NaN,0.0,0.0
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139000.0,0.0,31100.0,0.0,50000.0,0.0,107332.0,0.0,130826.0,0.0,165718.0,0.0,230500.0,0.0,NaN,NaN,176272.0,0.0,1344591.0,0.0,265000.0,0.0,155000.0,0.0,NaN,NaN
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,0.0,69076907.0,0.0,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,0.0,271004092.0,2809573.0,263194483.0,0.0,269253261.0,0.0,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,0.0,310639840.0,0.0,271692972.0,0.0
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,3245126.0,0.0,6186042.0,0.0,18423937.0,0.0,5624017.0,765306.0,7885069.0,0.0,15280397.0,1017509.0,16939495.0,348675.0,25973202.0,0.0,18074363.0,500409.0,16333100.0,0.0,17699567.0,0.0,18504048.0,0.0,18786627.0,0.0,35663228.0,0.0,49109105.0,0.0,26096928.0,6670328.0,32476153.0,1540154.0,19142494.0,0.0


In [20]:
gps_demo_donate[gps_demo_donate.filter(regex='(funding|pledge)20*', axis=1).isna().all(axis=1)]

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020
7,Bosnia Herzegovina,BIH,-0.247258,-0.125639,0.096463,0.429355,-0.11531,-0.193106,4.86,4.98,4.87,4.87,4.83,4.78,5.02,5.11,5.24,5.32,5.7,5.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### read in GDP in current US dollar data from WDI and merge with previous data frame

In [21]:
gdp = pd.read_excel('data/GDP_by_country_by_year.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
gdp = gdp.iloc[:,-18 : -1]
gdp = gdp.add_prefix('gdp')
gdp.reset_index()
gdp.head()


,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019
isocode,,,,,,,,,,,,,,,,,
ABW,2.021229e+09,2.228492e+09,2.330726e+09,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN
AFG,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
AGO,1.781271e+10,2.355205e+10,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,9.463542e+10
ALB,5.611492e+09,7.184681e+09,8.052076e+09,8.896074e+09,1.067732e+10,1.288135e+10,1.204422e+10,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527808e+10
AND,2.361727e+09,2.894922e+09,3.159905e+09,3.456442e+09,3.952601e+09,4.085631e+09,3.674410e+09,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09


In [22]:
gps_demo_donate_gdp = pd.merge(gps_demo_donate, gdp, how='left', on="isocode")
gps_demo_donate_gdp.shape

(76, 73)

In [23]:
# gps_demo_donate_gdp[gps_demo_donate_gdp.filter(regex='gdp.*', axis=1).isna().all(axis=1)]
gps_demo_donate_gdp.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,0.0,NaN,NaN,NaN,NaN,24000000.0,0.0,136493.0,0.0,2379910.0,0.0,3360.0,0.0,3000.0,0.0,2000.0,0.0,NaN,NaN,1000.0,0.0,NaN,NaN,NaN,NaN,3870542.0,0.0,903177.0,0.0,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,0.0,NaN,NaN,14769749.0,0.0,NaN,NaN,10000.0,0.0,10000.0,0.0,76432.0,0.0,10110696.0,0.0,16010000.0,7000000.0,NaN,NaN,2000000.0,0.0,10000.0,0.0,0.0,0.0,660000.0,0.0,10000.0,0.0,10000.0,0.0,NaN,NaN,0.0,0.0,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139000.0,0.0,31100.0,0.0,50000.0,0.0,107332.0,0.0,130826.0,0.0,165718.0,0.0,230500.0,0.0,NaN,NaN,176272.0,0.0,1344591.0,0.0,265000.0,0.0,155000.0,0.0,NaN,NaN,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,3.615580e+11,3.329765e+11,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.947493e+11,5.575314e+11,6.426959e+11,5.198715e+11,4.496634e+11
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,0.0,69076907.0,0.0,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,0.0,271004092.0,2809573.0,263194483.0,0.0,269253261.0,0.0,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,0.0,310639840.0,0.0,271692972.0,0.0,4.664881e+11,6.124904e+11,6.934078e+11,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,1.208847e+12,1.330136e+12,1.433904e+12,1.392681e+12
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69,3245126.0,0.0,6186042.0,0.0,18423937.0,0.0,5624017.0,765306.0,7885069.0,0.0,15280397.0,1017509.0,16939495.0,348675.0,25973202.0,0.0,18074363.0,500409.0,16333100.0,0.0,17699567.0,0.0,18504048.0,0.0,18786627.0,0.0,35663228.0,0.0,49109105.0,0.0,26096928.0,6670328.0,32476153.0,1540154.0,19142494.0,0.0,2.616958e+11,3.009042e+11,3.159744e+11,3.359986e+11,3.886914e+11,4.302943e+11,4.001723e+11,3.918927e+11,4.311203e+11,4.094252e+11,4.300687e+11,4.419961e+11,3.818176e+11,3.952277e+11,4.183162e+11,4.555083e+11,4.463147e+11


### adding columns of funding / GDP * 100 by year

In [24]:
df_funding = convert_percent('funding','gdp', gps_demo_donate_gdp)
df_funding.head()

,country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust,demo2019,demo2018,demo2017,demo2016,demo2015,demo2014,demo2013,demo2012,demo2011,demo2010,demo2008,demo2006,funding2003,pledge2003,funding2004,pledge2004,funding2005,pledge2005,funding2006,pledge2006,funding2007,pledge2007,funding2008,pledge2008,funding2009,pledge2009,funding2010,pledge2010,funding2011,pledge2011,funding2012,pledge2012,funding2013,pledge2013,funding2014,pledge2014,funding2015,pledge2015,funding2016,pledge2016,funding2017,pledge2017,funding2018,pledge2018,funding2019,pledge2019,funding2020,pledge2020,gdp2003,gdp2004,gdp2005,gdp2006,gdp2007,gdp2008,gdp2009,gdp2010,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,funding2003_gdp,funding2004_gdp,funding2005_gdp,funding2006_gdp,funding2007_gdp,funding2008_gdp,funding2009_gdp,funding2010_gdp,funding2011_gdp,funding2012_gdp,funding2013_gdp,funding2014_gdp,funding2015_gdp,funding2016_gdp,funding2017_gdp,funding2018_gdp,funding2019_gdp
0,Afghanistan,AFG,-0.201360,0.120764,0.289641,0.254712,0.166455,0.315964,2.85,2.97,2.55,2.55,2.77,2.77,2.48,2.48,2.48,2.48,3.02,3.06,NaN,NaN,NaN,NaN,500000.0,0.0,NaN,NaN,NaN,NaN,24000000.0,0.0,136493.0,0.0,2379910.0,0.0,3360.0,0.0,3000.0,0.0,2000.0,0.0,NaN,NaN,1000.0,0.0,NaN,NaN,NaN,NaN,3870542.0,0.0,903177.0,0.0,NaN,NaN,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.948438e+10,1.910135e+10,NaN,NaN,0.008053,NaN,NaN,0.237407,0.001097,0.015009,0.000019,0.000015,0.000010,NaN,0.000005,NaN,NaN,0.019865,0.004728
1,Algeria,DZA,0.059815,0.391530,-0.598255,0.254901,-0.009968,-0.177371,4.01,3.50,3.56,3.56,3.95,3.83,3.83,3.83,3.44,3.44,3.32,3.17,3178000.0,0.0,NaN,NaN,14769749.0,0.0,NaN,NaN,10000.0,0.0,10000.0,0.0,76432.0,0.0,10110696.0,0.0,16010000.0,7000000.0,NaN,NaN,2000000.0,0.0,10000.0,0.0,0.0,0.0,660000.0,0.0,10000.0,0.0,10000.0,0.0,NaN,NaN,0.0,0.0,6.786485e+10,8.532477e+10,1.031984e+11,1.170254e+11,1.349773e+11,1.710006e+11,1.372107e+11,1.612078e+11,2.000195e+11,2.090603e+11,2.097548e+11,2.138088e+11,1.659794e+11,1.600338e+11,1.673894e+11,1.737580e+11,1.699882e+11,0.004683,NaN,0.014312,NaN,0.000007,0.000006,0.000056,0.006272,0.008004,NaN,0.000953,0.000005,0.000000,0.000412,0.000006,0.000006,NaN
2,Argentina,ARG,-0.229308,0.041503,0.159679,-0.140457,0.027347,-0.206733,7.02,7.02,6.96,6.96,7.02,6.84,6.84,6.84,6.84,6.84,6.63,6.63,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139000.0,0.0,31100.0,0.0,50000.0,0.0,107332.0,0.0,130826.0,0.0,165718.0,0.0,230500.0,0.0,NaN,NaN,176272.0,0.0,1344591.0,0.0,265000.0,0.0,155000.0,0.0,NaN,NaN,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,3.615580e+11,3.329765e+11,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.947493e+11,5.575314e+11,6.426959e+11,5.198715e+11,4.496634e+11,NaN,0.000000,0.000000,0.000000,0.000000,0.000038,0.000009,0.000012,0.000020,0.000024,0.000030,0.000044,NaN,0.000032,0.000209,0.000051,0.000034
3,Australia,AUS,0.657004,0.137137,0.069660,0.022190,0.155331,0.298973,9.09,9.09,9.09,9.01,9.01,9.01,9.13,9.22,9.22,9.22,9.09,9.09,87736725.0,0.0,69076907.0,0.0,85073844.0,29859474.0,89757369.0,2262517.0,109371739.0,3817867.0,143674846.0,39494954.0,196311166.0,8733624.0,293856560.0,1923773.0,312711049.0,0.0,271004092.0,2809573.0,263194483.0,0.0,269253261.0,0.0,166236181.0,762236.0,205207886.0,2724017.0,243900706.0,2712522.0,248119692.0,0.0,310639840.0,0.0,271692972.0,0.0,4.664881e+11,6.124904e+11,6.934078e+11,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,1.208847e+12,1.330136e+12,1.433904e+12,1.392681e+12,0.018808,0.011278,0.012269,0.012031,0.012821,0.013631,0.021159,0.025639,0.022390,0.017528,0.016698,0.018348,0.012298,0.016976,0.018337,0.017304,0.022305
4,Austria,AUT,0.608285,-0.061829,0.161047,-0.055415,-0.035363,0.154628,8.29,8.29,8.42,8.41,8.54,8

### adding General government total expenditure as % of GDP & GDP per capita in current US dollar from IMF 

In [98]:
imf = pd.read_excel('data/IMF_Macro_Data_2020.xlsx').rename(columns={"ISO": "isocode"}).set_index('isocode')
imf_rename = imf.replace({'Gross domestic product per capita, current prices': 'gdpcapita', 'General government total expenditure': 'govexpense'})
imf_capita = imf_rename.query('`Subject Descriptor` == "gdpcapita"')
capita_filter = imf_capita.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('gdpcapita')

imf_expense = imf_rename.query('`Subject Descriptor` == "govexpense"')
expense_filter = imf_expense.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('govexpense')

expense_filter.head()


,govexpense2003,govexpense2004,govexpense2005,govexpense2006,govexpense2007,govexpense2008,govexpense2009,govexpense2010,govexpense2011,govexpense2012,govexpense2013,govexpense2014,govexpense2015,govexpense2016,govexpense2017,govexpense2018,govexpense2019
isocode,,,,,,,,,,,,,,,,,
AFG,11927,15069,15698,18303,21478,20918,21170,20801,21937,25028,24977,25402,25947,28029,27732,28936,27999
ALB,29680,30155,28892,29191,29309,31726,32679,29387,28923,28229,29193,31729,30377,29139,29148,28820,29383
DZA,32202,30765,27108,28853,33236,37950,42600,37251,40123,43543,36186,40606,45811,41665,39205,38166,38241
AGO,34400,29803,26527,30604,38129,52666,45007,39394,37402,37209,37050,36457,27051,22043,23806,19695,19223
ATG,26870,24476,24105,29327,27015,26418,36330,22563,23861,21077,23223,23010,26704,24658,23577,22985,22967


In [68]:
df_add_capita = pd.merge(df_funding, capita_filter, how='left', on="isocode")
# df_add_capita.shape
# df_add_capita[df_add_capita.filter(regex='gdpcapita.*', axis=1).isna().all(axis=1)]

df_add_expense = pd.merge(df_add_capita, expense_filter, how='left', on="isocode")
# df_add_expense.shape
# df_add_expense[df_add_expense.filter(regex='govexpense.*', axis=1).isna().all(axis=1)]



### adding region and income group data from WDI

In [73]:
region = pd.read_csv('data/WDICountry.csv').rename(columns={"Country Code": "isocode"}).set_index('isocode')
region_rename = region.rename({'Region': 'region', 'Income Group': 'income_type'}, axis=1)
region_filter = region_rename.filter(items=['region', 'income_type'], axis=1)
region_filter.head()


,region,income_type
isocode,,
ABW,Latin America & Caribbean,High income
AFG,South Asia,Low income
AGO,Sub-Saharan Africa,Lower middle income
ALB,Europe & Central Asia,Upper middle income
AND,Europe & Central Asia,High income


In [82]:
df_add_region = pd.merge(df_add_expense, region_filter, how='left', on="isocode")
# df_add_region.shape
# df_add_region[df_add_region.filter(items=['region', 'income_type'], axis=1).isna().any(axis=1)]

### adding population data by year from world bank

In [79]:
pop = pd.read_excel('data/Worldbank_Population_Data.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
pop_filter = pop.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('pop')
pop_filter.head()

,pop2003,pop2004,pop2005,pop2006,pop2007,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019
isocode,,,,,,,,,,,,,,,,,
ABW,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0
AFG,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0
AGO,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0
ALB,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0
AND,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0


In [84]:
df_add_pop = pd.merge(df_add_region, pop_filter, how='left', on="isocode")
# df_add_pop.shape
# df_add_pop[df_add_pop.filter(regex='pop.*', axis=1).isna().all(axis=1)]

### adding Net Official development assistance (ODA) data in current US dollar from WDI
### notice 54/76 countries have no ODA record from 2003 ~ 2019

In [85]:
oda = pd.read_excel('data/oda.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
oda_filter = oda.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('oda')
oda_filter.head()

,oda2003,oda2004,oda2005,oda2006,oda2007,oda2008,oda2009,oda2010,oda2011,oda2012,oda2013,oda2014,oda2015,oda2016,oda2017,oda2018,oda2019
isocode,,,,,,,,,,,,,,,,,
ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df_add_oda = pd.merge(df_add_pop, oda_filter, how='left', on="isocode")
# df_add_oda.shape
df_add_oda[df_add_oda.filter(regex='oda.*', axis=1).isna().all(axis=1)].shape

(54, 160)

In [93]:
#oda as % of GNI 
# odagni = pd.read_excel('data/oda gni.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
# odagni_filter = odagni.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('odagni')
# odagni_filter.head()
# df_add_odagni = pd.merge(df_add_oda, odagni_filter, how='left', on="isocode")
# df_add_odagni.shape
# df_add_odagni[df_add_odagni.filter(regex='odagni.*', axis=1).isna().all(axis=1)].shape

### adding net official aid recieve in current US dollar 
### notice 63/76 countries does not have any record

In [99]:

aid = pd.read_excel('data/offial aid received.xls', sheet_name='Data', skiprows=3).rename(columns={"Country Code": "isocode"}).set_index('isocode')
aid_filter = aid.filter(regex='^20(10|11|12|.*[3456789]$)', axis=1).add_prefix('aid')
aid_filter.head()

,aid2003,aid2004,aid2005,aid2006,aid2007,aid2008,aid2009,aid2010,aid2011,aid2012,aid2013,aid2014,aid2015,aid2016,aid2017,aid2018,aid2019
isocode,,,,,,,,,,,,,,,,,
ABW,7.622000e+07,-1.132000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
df_add_aid = pd.merge(df_add_oda, aid_filter, how='left', on="isocode")
# df_add_aid.shape
df_add_aid[df_add_aid.filter(regex='aid.*', axis=1).isna().all(axis=1)].shape

(63, 177)

### output

In [102]:
df_add_aid.to_csv('data/result.csv') 